<font size="10">
    <div align="center">
        Técnicas y algoritmos de búsqueda
    </div>
</font>
<font size="6">
    <div align="center">
        Informe de la práctica 3
    </div>
    <br>

</font>
<font size="5">
    <div align="center">
        Gabriel Niculescu Ruso 
    </div>
    <div align="center">
        Carlos Molera Canals
    </div>
    <div align="center">
        Jose Francisco Pérez Mompeán
    </div>
</font>
<font size="5">
    <div align="right">
        29/05/2025
    </div>
</font>


# Index
* [<font size="5">Introduction</font>](#introduction)
* [<font size="5">Alpha-Beta</font>](#Implementation)
* [<font size="5">Tabla de transposición</font>](#Entropy)
* [<font size="5">Función de evaluación</font>](#Automated-Clustering)
* [<font size="5">La red</font>](#Image-Segmentation)
# Introduction
[⬆️ Back to top](#index)